In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import math
from sklearn.metrics import accuracy_score
import random
import statistics
import skimage.io
import skimage.color
from numpy import unravel_index


In [ ]:
def transform(x):
  x=(x-np.mean(x,axis=0))/np.std(x,axis=0)
  #print(x.shape)
  return x

In [ ]:
# X, y = make_classification(n_samples=50000, n_features=20, n_informative=10, n_redundant=5,
#                            n_classes=2, weights=[0.7], class_sep=0.6, random_state=15)
b = np.load('/content/drive/MyDrive/bloodmnist.npz')
train_images=b['train_images']
val_images=b['val_images']
test_images=b['test_images']
Y_train=b['train_labels']
Y_test=b['val_labels']
Y_val=b['test_labels']
train_image=[]
test_image=[]
val_image=[]
for i in range(len(train_images)):
  train_image.append(skimage.color.rgb2gray(train_images[i]))
for i in range(len(test_images)):
  test_image.append(skimage.color.rgb2gray(test_images[i]))
for i in range(len(val_images)):
  val_image.append(skimage.color.rgb2gray(val_images[i]))
train_image=np.array(train_image)
test_image=np.array(test_image)
val_image=np.array(val_image)
X_train= train_image
X_val= test_image
X_test= val_image




In [ ]:
print(max(Y_train))

[7]


In [ ]:
def convolution(w,I,size):
  m,n=I.shape
  k1=size
  I1=np.zeros((m+k1-1,n+k1-1))
  k1=k1//2
  I1[k1:m+k1,k1:n+k1]=I
  I=I1.copy()
  I1=np.zeros((m+size-1,n+size-1))
  for i in range(k1,m+k1):
    for j in range(k1,n+k1):
      p=I[i-k1:i+k1+1,j-k1:j+k1+1]
      #print(i,j,k1)
      I1[i][j]=np.sum(np.sum(p*w))
  return I1[1:m+1,1:n+1]


In [ ]:
def convolution_backprop(w,I,size):
  m,n=I.shape
  k1=size
  I1=np.zeros((m+k1-1,n+k1-1))
  k1=k1//2
  I1[k1:m+k1,k1:n+k1]=I
  I=I1.copy()
  m=m+size-1
  n=n+size-1
  k1=w.shape[0]
  w1=np.zeros((size,size))
  for i in range(0,size):
    for j in range(0,size):
      p=I[i:i+k1,j:j+k1]
      w1[i][j]=np.sum(np.sum(p*w))
  return w1


In [ ]:
def sigmoid(z):
  if z < 0:
    return 1 - 1/(1 + math.exp(z))
  else:
    return 1/(1 + math.exp(-z))


In [ ]:
def grad_sigmoid(z):
  return sigmoid(z)*(1-sigmoid(z))

In [ ]:
def getindices(F,size,stride): # reprersent stride in maxpooling
  indexes=[]
  relu_outputs=[]
  for c in range(len(F)):
    img=np.zeros(F[c].shape)
    m,n=img.shape
    for i in range(0,m,stride):
      for j in range(0,n,stride):
        p=F[c][i:i+size,j:j+size]
        indices=unravel_index(p.argmax(), p.shape)
        # print(indices,int(indices[0]),int(indices[1]),i,j,img.shape)
        # if(len(indices)>2):
        #   indices=indices[0]
        # print(indices,len(indices))
        img[i+int(indices[0])][j+int(indices[1])]=1
        relu_outputs.append(max(0,p.max())) # relu is implemented at once 
    indexes.append(img)
  relu_outputs.append(1)
  return indexes,np.array(relu_outputs)


In [ ]:
# def softmax(l):
#   k=np.exp(l)
#   return k/np.sum(k)
def softmax(x):
    f = np.exp(x - np.max(x))  # shift values
    return f / f.sum(axis=0)

In [ ]:
def onehotencoding(a,max1):
  z=np.zeros(max1+1)
  z[a]=1
  return z 

In [ ]:
def forward_prop(kernel_Weights,img,size,stride,w):
  F=[]
  for i in range(len(kernel_Weights)):
    F.append(convolution(kernel_Weights[i],img,size))
  maxpool_indices,relu_outputs=getindices(F,size,stride)
  n=len(relu_outputs)
  # w=np.random.randn((n+1,2))
  L=softmax(np.dot(w.T,relu_outputs))
  # for i in range(len(L)):
  #   L[i]=softmax(L[i])

  return F,maxpool_indices,L,relu_outputs

In [ ]:
def backward_prop(I,F,maxpool_indices,kernel_weights,w_lastlayer,L,relu_outputs,stride,size,Y):
  w_new=w_lastlayer.copy()
  l=L
  l1=relu_outputs
  W=w_lastlayer
  # k=(l-Y)*l*(1-l) # error at Last layer
  k=l-Y
  # print(k.shape)
  delta=k
  dj_w=np.zeros_like(W)
  m=len(W)
  n=len(k)
  # for i in range(0,m):
  #   for j in range(0,n):
  #     dj_w[i][j]=k[j]*l1[i]
  l1_n=np.reshape(l1,(m,1))
  k_n=np.reshape(k,(n,1))
  dj_w=np.dot(l1_n,k_n.T)+W #l2 Regularizer

  # for i in range(0,m):
  #   dj_w[i]=k*l1[i]
  lambda1=0.001
  W=W-(lambda1*dj_w)
  w_new=W
  error_relu_nodes=[0]*len(relu_outputs) # error at relu output nodes
  for i in range(len(relu_outputs)):
    a=0
    # for j in range(len(k)):
    #   a+=k[j]*w_lastlayer[i][j]
    a=max(0,np.sum(k*w_lastlayer[i]))
    # print(k,w_lastlayer[i])
    # if math.isnan(a) or int(a)<=0:
    #   a=0
    error_relu_nodes[i]=a
  # now we need to find error of maxpool layer
  error_maxpool=maxpool_indices.copy()
  m1,n1=F[0].shape # size of image
  nodes_per_image=(m1//stride)*(n1//stride)
  for i in range(len(F)):
    nodes=error_relu_nodes[nodes_per_image*i:nodes_per_image*(i+1)]
    a=0
    for j in range(0,m1,stride):
      for y in range(0,n1,stride):
        error_maxpool[i][j:j+size,y:y+size]=nodes[a]*error_maxpool[i][j:j+size,y:y+size]
        a+=1
  # till now we calculated dj/dm  m= maxpool layer
  kernel_errors=[]
  for i in range(len(F)):
    X=error_maxpool[i]
    kernel_error=convolution_backprop(X,I,size)
    kernel_errors.append(kernel_error)
    kernel_weights[i]=kernel_weights[i]-(0.001)*(kernel_error)
  return kernel_weights,w_new


In [ ]:
kernel_weights=[]
epochs=3
stride=2
size=5
filters=8
#img_size=28*28
max_classes=int(max(Y_train))
n=(28//stride)*(28//stride)*filters+1
w_lastlayer=np.random.randn(n,max_classes)
for i in range(filters):
  kernel_weights.append(np.random.randn(size,size))

for i in range(epochs):
  for j in range(len(X_train)):
    Y = onehotencoding(Y_train[j],max_classes)
    F,maxpool_indices,L,relu_outputs= forward_prop(kernel_weights,X_train[i],size,stride,w_lastlayer)
    k_weights,w_new = backward_prop(X_train[i],F,maxpool_indices,kernel_weights,w_lastlayer,L,relu_outputs,stride,size,Y_train[j])
    kernel_weights=k_weights.copy()
    w_lastlayer=w_new
    # print(j,i)


In [ ]:
def accuracy(y_test,y_pred):
  count =0
  for i in range(len(y_pred)):
    if y_pred[i]==y_test[i]:
      count+=1
  accuracy = count*100/len(y_pred)
  return accuracy

In [ ]:
def predict(kernel_weights,x,size,stride,w_lastlayer):
  F,maxpool_indices,y,relu_outputs=forward_prop(kernel_weights,x,size,stride,w_lastlayer)
  return np.argmax(y)

In [ ]:
y_pred=[]
for i in range(len(X_test)):
  y_pred.append(predict(kernel_weights,X_test[i],size,stride,w_lastlayer)) #kernel_weights,X_train[i],size,stride,w_lastlayer
acc=accuracy(Y_test,y_pred)

In [ ]:
print(acc)

7.126168224299065


In [ ]:
y_pred=[]
for i in range(len(X_val)):
  y_pred.append(predict(kernel_weights,X_val[i],size,stride,w_lastlayer))
acc=accuracy(Y_val,y_pred)

In [ ]:
print(acc)

In [ ]:
y_pred=[]
for i in range(len(X_train)):
  y_pred.append(predict(kernel_weights,X_train[i],size,stride,w_lastlayer))
acc=accuracy(Y_train,y_pred)
print(acc)